# **PROYECTO**

## COSMIC DEFENDER

In [1]:
import pygame 
import random 
import time 

pygame 2.1.2 (SDL 2.0.20, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
pygame.init() #Se inicia el constructor de pygame
pygame.mixer.init() #Se inicia el constructor de sonido de pygame.

## Configuraciones pantalla de inicio

In [3]:
anchoPantalla=800 
altoPantalla=600 

pantalla=pygame.display.set_mode((anchoPantalla,altoPantalla))
pygame.display.set_caption("Cosmic Defenders")

pantallaDeInicio=pygame.image.load("Inicio-PrimerNivel.png").convert()
pantallaDeInicio=pygame.transform.scale(pantallaDeInicio,(800,600))

fuenteTitulo=pygame.font.Font("Silkscreen-Bold.ttf",50)
textoTitulo=fuenteTitulo.render("Cosmic Defenders",True,(0,0,0))
textoRect1=textoTitulo.get_rect(center=(anchoPantalla // 2,altoPantalla // 6))

fuenteBotones=pygame.font.Font("Silkscreen-Bold.ttf",40)
textoInicio=fuenteBotones.render("INICIO",True,(255,255,255))
textoRect2=textoInicio.get_rect(center=(anchoPantalla // 2,altoPantalla // 1.5))
inicio=pygame.Rect(textoRect2.left - 1,textoRect2.top - 1,textoRect2.width + 1,textoRect2.height + 1)

textoAjustes=fuenteBotones.render("AJUSTES",True,(255,255,255))
textoRect3=textoAjustes.get_rect(center=(anchoPantalla // 2,altoPantalla // 1.3))
ajustes=pygame.Rect(textoRect3.left - 1,textoRect3.top - 1,textoRect3.width + 1,textoRect3.height + 1)

musicaInicio=pygame.mixer.music.load("Himno Universidad de Santiago De Chile.mp3")
reproducirMusica=pygame.mixer.music.play(-1)

sonidoDisparo=pygame.mixer.Sound("piun.mp3")
sonidoDisparoJefe=pygame.mixer.Sound("piuJefe.mp3")
sonidoJefeActivo=True

musicaNiveles={
    1: "primer_nivel.mp3",
    2: "segundo_nivel.mp3",
    3: "tercer_nivel.mp3",
    4: "ultimo_nivel.mp3"
}

## Clases

In [4]:
class Jugador(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.imagen=pygame.image.load("Nave.png").convert_alpha()
        self.imagen=pygame.transform.scale(self.imagen,(50,50))
        self.rect=self.imagen.get_rect()
        self.rect.center=(anchoPantalla // 2,altoPantalla - 50)
        self.velocidad=15
        self.proyectiles=pygame.sprite.Group()
        self.enemigosDerribados=0
        self.vida=5
        self.escudo=1
        self.daño=1

    def mover(self,teclas):
        """
        Funcion para mover al jugador en el eje X solamente

        Parametros:
        ----------
        teclas = variable que representa las teclas del teclado, se puede asociar a una unica tecla
        """
        if teclas[pygame.K_LEFT] and self.rect.left > 0:
            self.rect.x -= self.velocidad
            
        if teclas[pygame.K_RIGHT] and self.rect.right < anchoPantalla:
            self.rect.x += self.velocidad

    def aparicion(self,pantalla):
        """
        Funcion que permite mostrar al jugador en pantalla

        Parametros:
        ------------------
        pantalla = Variable que representa la pantalla del videojuego
        """
        pantalla.blit(self.imagen,self.rect)

    def disparar(self):
        """
        Funcion que permite al jugador disparar proyectiles
        guarda los proyectiles en una lista.
        """
        proyectil=Proyectil(self.rect.centerx,self.rect.top)
        self.proyectiles.add(proyectil)

    def recibirDaño(self):
        """
        Funcion que representa el daño recibido al jugador
        lo que provoca que baje la vida del mismo o el escudo si no se tiene vida
        """
        self.vida-=1
        if self.vida < 0:
            self.escudo-=1
        
    def actualizar(self,meteoritos,puntaje):
        """
        Funcion que representa las interacciones de los enemigos con diferentes instancias
        
        Parametros:
        ----------------
        meteoritos (list) = Variable que representa a una lista de instancias de Enemigos 
        puntaje (int) = Variable que representa el puntaje obtenido del jugador 

        Retorna
        ---------------
        puntaje (int) = variable de puntaje obtenido del jugador actualizado
        """
        # interaccion meteorito-proyectil
        for proyectil in self.proyectiles:
            proyectil.mover()
            proyectil.aparecer(pantalla)
            for meteorito in meteoritos:
                if proyectil.rect.colliderect(meteorito.rect):
                    if isinstance(meteorito, Enemigos):
                        puntosObtenidos=meteorito.recibirDisparo(self)
                        puntaje+=puntosObtenidos
                        proyectil.kill()
                        self.enemigosDerribados+=1
                        
        # interaccion meteorito-jugador
        for meteorito in meteoritos:
            if isinstance(meteorito,Enemigos):
                if self.rect.colliderect(meteorito.rect):
                    self.recibirDaño()
                    meteorito.kill() 
                
        return puntaje
        
    def actualizarJefe(self,puntaje,jefe,jefeActivo):
        """
        Función que crea las interacciones entre el jefe final y distintas instancias

        Parametros:
        -----------------
        puntaje (int) = Variable que representa el puntaje actual del jugador
        jefe = Instancia de la clase JefeFinal
        jefeActivo (bool) = Booleano que representa si el jefe ha aparecido

        Retorna:
        ----------------
        puntaje (int) = Variable que representa el puntaje obtenido actualizado del jugador
        """
        
        # Interaccion jefe-proyectil
        for proyectil in self.proyectiles:
            proyectil.mover()
            proyectil.aparecer(pantalla)

            if isinstance(jefe,JefeFinal) and jefeActivo == True and proyectil.rect.colliderect(jefe.rect):
                puntosObtenidos=jefe.recibirDisparo(self)
                puntaje+=puntosObtenidos
                proyectil.kill()
                
       # interaccion proyectil del jefe - Jugador
        for proyectilJefe in jefe.proyectiles:
            if self.rect.colliderect(proyectilJefe.rect):
                self.recibirDaño()
                proyectilJefe.kill()
                
        return puntaje
    
    def actualizarMejora(self,mejora):
        """
        Funcion que crea la interaccion de las mejoras con el jugador
        """
        if isinstance(mejora,Mejora):
            if self.rect.colliderect(mejora.rect):
                mejora.modificacion(self)  
                mejora.kill() 


    def mostrarBarraVida(self,pantalla):
        """
        Funcion para mostrar la barra de vida junto al simbolo del mismo
        """
        largoBarraVida=200
        altoBarraVida=20
        
        largoActualVida=(self.vida/5)*largoBarraVida

        vidaCompleta=pygame.image.load("coralleno.png")
        vidaCompleta=pygame.transform.scale(vidaCompleta,(20,20))
        pantalla.blit(vidaCompleta,(10,10))       
        pygame.draw.rect(pantalla,(250,0,0),(30,10,largoBarraVida,altoBarraVida))

        if self.vida == 0:
            vidaVacia=pygame.image.load("coravacio.png")
            vidaVacia=pygame.transform.scale(vidaVacia,(20,20))
            pantalla.blit(vidaVacia,(10,10))
        
        pygame.draw.rect(pantalla,(0,255,0),(30,10,largoActualVida,altoBarraVida))

    def mostrarBarraEscudo(self,pantalla):
        """
        Función para mostrar la barra de escudo de la nave junto a su simbolo.
        """
        largoBarraEscudo=50
        altoBarraEscudo=20
        
        largoActualEscudo=(self.escudo)*largoBarraEscudo

        escudo=pygame.image.load("escudo.png")
        escudo=pygame.transform.scale(escudo,(20,20))
        pantalla.blit(escudo,(10,32))
        pygame.draw.rect(pantalla,(250,0,0),(30,32,largoBarraEscudo,altoBarraEscudo))
        
        pygame.draw.rect(pantalla,(255,215,0),(30,32,largoActualEscudo,altoBarraEscudo))

class Proyectil(pygame.sprite.Sprite):
    def __init__(self,x,y):
        super().__init__()
        self.imagen=pygame.image.load("proyectil.png").convert_alpha()
        self.imagen=pygame.transform.scale(self.imagen,(10,30))
        self.rect=self.imagen.get_rect()
        self.rect.centerx=x
        self.rect.bottom=y
        self.velocidad=7

    def mover(self):
        """
        Función para disparar los proyectiles de la nave verticalmente hacia arriba.
        """
        self.rect.y-=self.velocidad 

        if self.rect.bottom < 0:
            self.kill()

    def aparecer(self,pantalla):
        """
        Función para mostrar los proyectiles de la nave en pantalla.

        Parametros:
        -----------
        pantalla = Variable que representa la pantalla del videojuego
        """
        pantalla.blit(self.imagen, self.rect)
    
class Enemigos(pygame.sprite.Sprite):
    def __init__(self,enemigos,velocidad,tamano,disparosNecesarios,puntaje):
        super().__init__()
        self.imagen=pygame.image.load(enemigos).convert_alpha()
        self.imagen=pygame.transform.scale(self.imagen,tamano)
        self.rect=self.imagen.get_rect()
        self.velocidad=velocidad
        self.disparosRecibidos=0
        self.disparosNecesarios=disparosNecesarios 
        self.puntaje=puntaje

    def caer(self):
        """
        Función para que los meteoritos caigan verticalmente hacia abajo.
        """
        self.rect.y+=self.velocidad
        if self.rect.y > altoPantalla:
            self.kill()

    def mostrar(self,pantalla):
        """
        Función para mostrar los meteoritos en pantalla.

        Parametros:
        -----------
        pantalla = Variable que representa la pantalla del videojuego
        """
        pantalla.blit(self.imagen,self.rect)

    def recibirDisparo(self,jugador):
        """
        Función para que los meteoritos reciban daño y se destruyan.

        Parametros:
        -----------
        jugador = Instancia de Jugador

        Retorna:
        ----------
        self.puntaje (Int) = Variable que representa el puntaje del tipo de meteorito que es derrotado
        """
        self.disparosRecibidos+=jugador.daño
        if self.disparosRecibidos >= self.disparosNecesarios:
            self.kill()
            return self.puntaje
        return 0

class Meteorito1(Enemigos):
    def __init__(self):
        super().__init__("meteorito1.png",velocidad=1, tamano=(50,70),disparosNecesarios=1,puntaje=10)
        x=random.randint(0,anchoPantalla-70)
        y=35
        self.rect=pygame.Rect(x,y,50,70)

class Meteorito2(Enemigos):
    def __init__(self):
        super().__init__("meteorito2.png",velocidad=2,tamano=(50,70),disparosNecesarios=2,puntaje=20)
        x=random.randint(0,anchoPantalla-70)
        y=35
        self.rect=pygame.Rect(x,y,50,70)

class Meteorito3(Enemigos):
    def __init__(self):
        super().__init__("meteorito3.png",velocidad=1,tamano=(120,150),disparosNecesarios=4,puntaje=40)
        x=random.randint(0,anchoPantalla-150)
        y=35
        self.rect=pygame.Rect(x,y,120,150)    
      
class JefeFinal(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.imagen=pygame.image.load("Jefe_final.png").convert_alpha()
        self.imagen=pygame.transform.scale(self.imagen,(200,200))
        self.rect=self.imagen.get_rect()
        self.rect.center=(anchoPantalla // 2,150)
        self.velocidad=3
        self.direccion=1
        self.disparosNecesarios=10
        self.disparosRecibidos=0
        self.puntaje=500
        self.proyectiles=pygame.sprite.Group()
        self.tiempoUltimoDisparo=0

    def mover(self):
        """
        Función para que el jefe final se mueva horizontalmente con limites en los costados.
        """
        self.rect.x+=self.velocidad

        if self.rect.right > anchoPantalla:
            self.rect.right=anchoPantalla
            self.velocidad *= -1

        if self.rect.left < 0:
            self.rect.left=0
            self.velocidad *= -1

    def recibirDisparo(self,jugador):
        """
        Función para que el jefe reciba daño.

        Parametros:
        -----------
        jugador = Instancia de Jugador

        Retorna:
        ----------
        self.puntaje (Int) = Variable que representa le puntaje obtenido por derrotar al jefe 
        """

        self.disparosRecibidos+=jugador.daño
        if self.disparosRecibidos >= self.disparosNecesarios:
            self.kill()
            return self.puntaje
        return 0
    
    def disparar(self):
        """
        Función para que el jefe realize los disparos.
        """
        if pygame.time.get_ticks()-self.tiempoUltimoDisparo > 500 and random.randrange(0,1000,1) <= 50:  
            proyectilJefeIzquierda=ProyectilJefe(self.rect.centerx-30,self.rect.bottom)
            proyectilJefeDerecha=ProyectilJefe(self.rect.centerx+30,self.rect.bottom)
            
            if sonidoJefeActivo:
                sonidoDisparoJefe.play()
                
            self.proyectiles.add(proyectilJefeIzquierda)
            self.proyectiles.add(proyectilJefeDerecha)
            self.tiempoUltimoDisparo=pygame.time.get_ticks()
            
    def mostrar(self,pantalla):
        """
        Función para mostrar al jefe final en pantalla.

        Parametros:
        -----------
        pantalla = Variable que representa la pantalla del videojuego
        """
        pantalla.blit(self.imagen,self.rect)
        
    def actualizarProyectiles(self,pantalla):
        """
        Función para actualizar y mostrar los proyectiles disparados por el jefe.

        Parametros:
        -----------
        pantalla = Variable que representa la pantalla del videojuego
        """
        for proyectil in self.proyectiles:
            proyectil.mover()
            proyectil.aparecer(pantalla)

class ProyectilJefe(pygame.sprite.Sprite):
    def __init__(self,x,y):
        super().__init__()
        self.imagen=pygame.image.load("proyectil_jefe.png").convert_alpha()
        self.imagen=pygame.transform.scale(self.imagen,(20,40))
        self.rect=self.imagen.get_rect()
        self.rect.centerx=x
        self.rect.top=y
        self.velocidad=5

    def mover(self):
        """
        Función para que los pryectiles se muevan verticalmente hacia abajo.
        """
        self.rect.y+=self.velocidad
        if self.rect.top > altoPantalla:
            self.kill()

    def aparecer(self,pantalla):
        """
        Función para mostrar los proyectiles en pantalla.

        Parametros:
        -----------
        pantalla = Variable que representa la pantalla del videojuego  
        """
        pantalla.blit(self.imagen,self.rect)

class Mejora(pygame.sprite.Sprite):
    def __init__(self, imagen, escalado):
        super().__init__()
        self.x=random.randint(0,700)  
        self.y=random.randint(-10,-10)  
        self.image=pygame.image.load(imagen).convert_alpha() 
        self.image=pygame.transform.scale(self.image,escalado)
        self.rect=self.image.get_rect(center=(self.x,self.y))
        self.velocidad=5  

    def caer(self):
        """
        Funcion que representa el movimiento vertical de la mejora 
        """

        self.rect.y+=self.velocidad
        if self.rect.top > altoPantalla:  
            self.kill()

    def modificacion(self,jugador):
        """
        Funcion que tendra como fin la mejora de atributos hacia el jugador
        """
        pass
    
    def mostrar(self,pantalla):
        """ 
        Función para mostrar la mejora en pantalla.

        Parametros:
        -----------
        pantalla = Variable que representa la pantalla del videojuego
        """
        pantalla.blit(self.image,self.rect) 

class MejoraVida(Mejora):
    def __init__(self):
        super().__init__("coralleno.png",(40,40))  

    def modificacion(self,jugador):
        """
        Funcion que aumenta la vida del jugador
        """
        jugador.vida+=1  


class MejoraEscudo(Mejora):
    def __init__(self):
        super().__init__("escudo.png",(40,40))  

    def modificacion(self,jugador):
        """
        Funcion que aumenta el escudo del jugador
        """
        jugador.escudo+=1  


class MejoraDaño(Mejora):
    def __init__(self):
        super().__init__("mejoraBalas.png",(10,50))  

    def modificacion(self,jugador):
        """
        Funcion que aumenta el daño que provoca el jugador
        """
        jugador.daño+=1  


## Funciones

In [5]:
def pantallaAjustes():
    """
    Función para definir la pantalla de ajustes.

    Retorna:
    -------------------------------------------------------------------
    textoRectSalir = Variable para salir de la pantalla de ajustes.
    textoRectOnMusica = Variable para encender la música.
    textoRectOffMusica = Variable para apagar la música.
    textoRectOnSonido = Variable para encender el sonido del juego.
    textoRectOffSonido = Variable para apagar el sonido del juego.
    """

    pantalla.fill((0,0,0))

    fuenteAjustes=pygame.font.Font("Silkscreen-Bold.ttf",50)
    textoMusica=fuenteAjustes.render("Musica",True,(255,255,255))
    textoRectMusica=textoMusica.get_rect(center=(anchoPantalla // 2,altoPantalla // 5))

    fuenteOnMusica=pygame.font.Font("Silkscreen-Bold.ttf",30)
    textoOnMusica=fuenteOnMusica.render("ON",True,(255,255,255))
    textoRectOnMusica=textoOnMusica.get_rect(center=(anchoPantalla // 2.3,altoPantalla // 3))
    pygame.draw.rect(pantalla,(0,255,0),textoRectOnMusica)

    fuenteOffMusica=pygame.font.Font("Silkscreen-Bold.ttf",30)
    textoOffMusica=fuenteOffMusica.render("OFF",True,(255,255,255))
    textoRectOffMusica=textoOffMusica.get_rect(center=(anchoPantalla // 1.7,altoPantalla // 3))
    pygame.draw.rect(pantalla,(255,0,0),textoRectOffMusica)

    fuenteSonido=pygame.font.Font("Silkscreen-Bold.ttf",50)
    textoSonido=fuenteSonido.render("Sonidos",True,(255,255,255))
    textoRectSonido=textoSonido.get_rect(center=(anchoPantalla // 2,altoPantalla // 2))

    fuenteOnSonido=pygame.font.Font("Silkscreen-Bold.ttf",30)
    textoOnSonido=fuenteOnSonido.render("ON", True, (255, 255, 255))
    textoRectOnSonido=textoOnSonido.get_rect(center=(anchoPantalla // 2.3,altoPantalla // 1.6))
    pygame.draw.rect(pantalla,(0,255,0),textoRectOnSonido)

    fuenteOffSonido=pygame.font.Font("Silkscreen-Bold.ttf",30)
    textoOffSonido=fuenteOffSonido.render("OFF",True,(255,255,255))
    textoRectOffSonido=textoOffSonido.get_rect(center = (anchoPantalla // 1.7,altoPantalla // 1.6))
    pygame.draw.rect(pantalla,(255,0,0),textoRectOffSonido)

    fuenteSalir=pygame.font.Font("Silkscreen-Bold.ttf",50)
    textoSalir=fuenteSalir.render("Salir",True,(255,255,255))
    textoRectSalir=textoSalir.get_rect(center=(anchoPantalla // 2,altoPantalla // 1.3))

    pantalla.blit(textoMusica,textoRectMusica)
    pantalla.blit(textoOnMusica,textoRectOnMusica)
    pantalla.blit(textoOffMusica,textoRectOffMusica)
    pantalla.blit(textoSonido,textoRectSonido)
    pantalla.blit(textoOnSonido,textoRectOnSonido)
    pantalla.blit(textoOffSonido,textoRectOffSonido)
    pantalla.blit(textoSalir,textoRectSalir)

    return textoRectSalir,textoRectOnMusica,textoRectOffMusica,textoRectOnSonido,textoRectOffSonido

def pantallaPerdida():
    """
    Función para definir la pantalla de perdida.

    Retorna:
    -------------------------------------------------------------------
    textoRectSalir = Variable para salir de la pantalla de ajustes.
    textoRectOnMusica = Variable para encender la música.
    textoRectOffMusica = Variable para apagar la música.
    textoRectOnSonido = Variable para encender el sonido del juego.
    textoRectOffSonido = Variable para apagar el sonido del juego.
    """
    pantalla.fill((0,0,0))

    fuenteDerrota=pygame.font.Font("Silkscreen-Bold.ttf",50)
    textoDerrota=fuenteDerrota.render("Has perdido",True,(255,255,255))
    textoRectDerrota=textoDerrota.get_rect(center=(anchoPantalla // 2,altoPantalla // 5))
    
    fuenteSalir=pygame.font.Font("Silkscreen-Bold.ttf",50)
    textoSalir=fuenteSalir.render("Salir",True,(255,255,255))
    textoRectSalir=textoSalir.get_rect(center=(anchoPantalla // 2,altoPantalla // 1.2))

    fuenteReiniciar=pygame.font.Font("Silkscreen-Bold.ttf",50)
    textoReiniciar=fuenteReiniciar.render("Reiniciar",True,(255,255,255))
    textoRectReiniciar=textoReiniciar.get_rect(center=(anchoPantalla // 2,altoPantalla // 1.5))

    pantalla.blit(textoDerrota,textoRectDerrota)
    pantalla.blit(textoSalir,textoRectSalir)
    pantalla.blit(textoReiniciar,textoRectReiniciar)

    pygame.display.update()

    return textoRectReiniciar,textoRectSalir
    
def mostrarPuntaje(pantalla,puntaje):
    """
    Función para mostrar el puntaje en pantalla.
    ----------------------------------------------------------------
    Parametros:
    pantalla = Varible que representa la pantalla del juego. 
    puntaje = Variable que representa el puntaje actual del jugador.
    """

    fuentePuntaje=pygame.font.Font("Silkscreen-Bold.ttf",22)
    textoPuntaje=fuentePuntaje.render(f"Puntuación: {puntaje}",True,(255,255,255))
    pantalla.blit(textoPuntaje,(anchoPantalla-300,10))
    
def reproducirMusica(nivel):
    """
    Función para cargar y reproducir música del nivel.

    Parametros:
    -----------
    nivel (Int) = Variable que representa el nivel actual del jugador
    """
    if nivel in musicaNiveles:
        pygame.mixer.music.stop()
        pygame.mixer.music.load(musicaNiveles[nivel])
        pygame.mixer.music.play(-1)
 
meteoritos=[]  
crearMeteoritos=0  
tiempoDeCreacion=5500  
cantidadDeMeteoritos=2000  
contadorDeMeteoritos=0  
jefeActivo=False
mejoraVidaActivo=False
mejoraEscudoActivo=False
mejoraDañoActivo=False

def enemigosPorNivel(niveles,meteoritos,crearMeteoritos,tiempoDeCreacion,cantidadDeMeteoritos,contadorDeMeteoritos,jugador,nivelCambiado,jefe,
                     jefeActivo,mejoraVidaActivo,mejoraEscudoActivo,mejoraDañoActivo):
    """
    Función para generar enemigos por cada nivel del juego y controlar eventos del juego.

    Parametros:
    ---------------------------------------------------------------------------------------
        niveles (Int) = Nivel actual.
        meteoritos (List) = Lista para almacenar los meteoritos.
        crearMeteoritos (Int) = Controlador del tiempo para crear meteoritos.
        tiempoDeCreacion (Int) = Intervalo en milisegundos para crear meteoritos.
        cantidadDeMeteoritos (Int) = Número total de meteoritos por nivel.
        contadorDeMeteoritos (Int) = Contador para saber cuántos meteoritos han sido generados.
        jugador = Instancia de jugador.
        nivelCambiado (Bool) =  Booleano que representa el cambio de nivel
        jefe = Instancia de JefeFinal
        jefeActivo (Bool) = Booleano que representa si el jefe ha aparecido
        mejoraVidaActivo (Bool) = Booleano que representa si ha aparecido una mejora de salud
        mejoraEscudoActivo (Bool) = Booleano que representa si ha aparecido una mejora de escudo
        mejoraDañoActivo (Bool) = Booleano que representa si ha aparecido una mejora de daño

    Retorna:
    -----------------------------------------------------------------------------------------
        meteoritos (List) = Lista de meteoritos.
        crearMeteoritos (Int) = Entero que contiene el tiempo de creación de los meteoritos. 
        contadorDeMeteoritos (Int) = Entero que contiene la cantidad de meteoritos creados.
        niveles (Int) = Entero que contiene el nivel del juego.
        nivelCambiado (Bool) = Booleano que contiene el cambio de nivel.
        jefeActivo (Bool) = Booleano que representa si el jefe ha aparecido
        mejoraVidaActivo (Bool) = Booleano que representa si ha aparecido una mejora de salud
        mejoraEscudoActivo (Bool) = Booleano que representa si ha aparecido una mejora de escudo
        mejoraDañoActivo (Bool) = Booleano que representa si ha aparecido una mejora de daño

    """
    tiempoNivel=pygame.time.get_ticks()
    #Condicionales para crear meteoritos por cada nivel.
    if (tiempoNivel-crearMeteoritos >= tiempoDeCreacion) and (contadorDeMeteoritos < cantidadDeMeteoritos):
        if niveles == 1:
            filaDeMeteoritos=5
            for i in range(filaDeMeteoritos):
                meteoritos.add(Meteorito1())
        
            if jugador.enemigosDerribados >= 5:
                niveles+=1
                jugador.enemigosDerribados=0
                nivelCambiado=True
                meteoritos.empty()

        elif niveles == 2:
            filaDeMeteoritos=6
            for i in range(filaDeMeteoritos):
                meteoritos.add(Meteorito1())  

            if jugador.enemigosDerribados >= 5: 
                niveles+=1  
                jugador.enemigosDerribados=0 
                nivelCambiado=True
                meteoritos.empty()

        elif niveles == 3:
            filaDeMeteoritos=4
            for i in range(filaDeMeteoritos):
                meteoritos.add(Meteorito1())  

            if jugador.enemigosDerribados >= 5:  
                niveles+=1 
                jugador.enemigosDerribados=0
                nivelCambiado=True
                meteoritos.empty()
                
            if mejoraVidaActivo == False:
                mejora=MejoraVida()
                meteoritos.add(mejora)
                mejoraVidaActivo=True

        elif niveles == 4:
            filaDeMeteoritos=4
            for i in range(filaDeMeteoritos):
                # Se introduce a un tipo de meteorito nuevo
                # El cual aparece a traves de un numero random
                # Si el numero random es menor a un numero, este aparecera
                numeroRandom=random.randrange(0,100,1)
                if numeroRandom < 30:
                    meteoritos.add(Meteorito2())  
                else:
                    meteoritos.add(Meteorito1())
                    
            if jugador.enemigosDerribados >= 5:  
                niveles+=1  
                jugador.enemigosDerribados=0
                nivelCambiado=True
                meteoritos.empty()

        elif niveles == 5:
            filaDeMeteoritos=4
            for i in range(filaDeMeteoritos):
                numeroRandom=random.randrange(0,100,1)
                if numeroRandom < 50:
                    meteoritos.add(Meteorito2())  
                else:
                    meteoritos.add(Meteorito1())
                    
            if jugador.enemigosDerribados >= 5:  
                niveles+=1 
                jugador.enemigosDerribados=0
                nivelCambiado=True
                meteoritos.empty()

        elif niveles == 6:
            filaDeMeteoritos=4
            for i in range(filaDeMeteoritos):
                numeroRandom=random.randrange(0,100,1)
                if numeroRandom < 60:
                    meteoritos.add(Meteorito2())  
                else:
                    meteoritos.add(Meteorito1())
                    
            if jugador.enemigosDerribados >= 5:  
                niveles+=1  
                jugador.enemigosDerribados=0
                nivelCambiado=True
                meteoritos.empty()
            if mejoraEscudoActivo == False:
                mejora=MejoraEscudo()
                meteoritos.add(mejora)
                mejoraEscudoActivo=True

        elif niveles == 7:
            filaDeMeteoritos=4
            for i in range(filaDeMeteoritos):
                # Aparece otro tipo de enemigo nuevo
                # Este tendra una posibilidad de aparecer igual al enemigo anterior
                numeroRandom=random.randrange(0,100,1)
                numeroRandom2=random.randrange(0,100,1)
                if numeroRandom < 60:
                    meteoritos.add(Meteorito2())  
                if numeroRandom2 < 30:
                    meteoritos.add(Meteorito3())
                else:
                    meteoritos.add(Meteorito1())
                    
            if jugador.enemigosDerribados >= 5:  
                niveles+=1  
                jugador.enemigosDerribados=0
                nivelCambiado=True
                meteoritos.empty()

        elif niveles == 8:
            filaDeMeteoritos=4
            for i in range(filaDeMeteoritos):
                numeroRandom=random.randrange(0,100,1)
                numeroRandom2=random.randrange(0,100,1)
                if numeroRandom < 60:
                    meteoritos.add(Meteorito2())  
                if numeroRandom2 < 40:
                    meteoritos.add(Meteorito3())
                else:
                    meteoritos.add(Meteorito1())
                    
            if jugador.enemigosDerribados >= 5:  
                niveles+=1  
                jugador.enemigosDerribados=0
                nivelCambiado=True
                meteoritos.empty()

        elif niveles == 9:
            filaDeMeteoritos=4
            for i in range(filaDeMeteoritos):
                numeroRandom=random.randrange(0,100,1)
                numeroRandom2=random.randrange(0,100,1)
                if numeroRandom < 60:
                    meteoritos.add(Meteorito2())  
                if numeroRandom2 < 50:
                    meteoritos.add(Meteorito3())
                else:
                    meteoritos.add(Meteorito1())
                    
                    
            if jugador.enemigosDerribados >= 10:  
                niveles+=1  
                jugador.enemigosDerribados=0
                nivelCambiado=True
                meteoritos.empty()
                
            if mejoraDañoActivo == False:
                mejora=MejoraDaño()
                meteoritos.add(mejora)
                mejoraDañoActivo=True

                
        elif niveles == 10:  # Si llegamos al nivel 10, agregamos el jefe final
            filaDeMeteoritos=4
            for i in range(filaDeMeteoritos):
                numeroRandom=random.randrange(0,100,1)
                numeroRandom2=random.randrange(0,100,1)
                if numeroRandom < 50:
                    meteoritos.add(Meteorito2())  
                if numeroRandom2 < 50:
                    meteoritos.add(Meteorito3())
                else:
                    meteoritos.add(Meteorito1())
            if jefeActivo == False: # Se verifica que no este el jefe
                meteoritos.add(jefe)
                jefeActivo=True 


        contadorDeMeteoritos+=filaDeMeteoritos #Se actualiza el valor del contador de meteoritos.
        crearMeteoritos=tiempoNivel #Se actualiza el tiempo de creación de los meteoritos.

    return meteoritos,crearMeteoritos,contadorDeMeteoritos,niveles,nivelCambiado,jefeActivo,mejoraVidaActivo,mejoraEscudoActivo,mejoraDañoActivo

def mostrarMensaje(pantalla,mensaje,color,pos_x,pos_y,font_size=36):
    """
    Función para mostrar un mensaje en la pantalla.

    Parametros:
    -----------------------
    pantalla = Variable que representa la pantalla del videojuego
    mensaje (Str) = String que sera el mensaje a mostrar
    color = Variable que representa el color del mensaje modificado por pygame
    pos_x = Posicion x del mensaje
    pos_y = Posicion y del mensaje
    font_size (Int) = Variable que representa el tamano del mensaje (por defecto 36) 
    """

    pantalla.fill((0,0,0))
    fuente=pygame.font.Font("Silkscreen-Bold.ttf",font_size)
    texto=fuente.render(mensaje,True,color)
    rect=texto.get_rect(center=(pos_x,pos_y))
    pantalla.blit(texto,rect)
    pygame.display.flip()

def reproducirMusica(nivel):
    """
    Reproduce la música correspondiente al nivel.

    Parametros:
    -------------
    nivel (Int) = Variable que representa el nivel actual del jugador
    """
    pygame.mixer.music.stop()

    if nivel >= 1 and nivel < 4:
        archivoMusica="primer_nivel.mp3"   
    elif nivel >= 4 and nivel < 7:
        archivoMusica="segundo_nivel.mp3"  
    elif nivel >= 7 and nivel < 10:
        archivoMusica="tercer_nivel.mp3"
    else:
        archivoMusica="ultimo_nivel.mp3"
        
    pygame.mixer.music.load(archivoMusica)
    pygame.mixer.music.play(-1)  

def cambiarPantallaNiveles(nivel):
    """
    Función para cambiar el fondo según el nivel.

    Parametros:
    ----------
    nivel (Int) = Variable que repsenta el nivel actual del jugador
    """
    pantallaNiveles1=pygame.image.load("Inicio-PrimerNivel.png")
    pantallaNiveles2=pygame.image.load("Niveles(4-6).png")
    pantallaNiveles3=pygame.image.load("Niveles(7-9).png")
    pantallaNivelFinal=pygame.image.load("NivelFinal.png")

    pantallaNiveles1=pygame.transform.scale(pantallaNiveles1,(anchoPantalla,altoPantalla))
    pantallaNiveles2=pygame.transform.scale(pantallaNiveles2,(anchoPantalla,altoPantalla))
    pantallaNiveles3=pygame.transform.scale(pantallaNiveles3,(anchoPantalla,altoPantalla))
    pantallaNivelFinal=pygame.transform.scale(pantallaNivelFinal,(anchoPantalla,altoPantalla))

    if nivel >= 1 and nivel < 4:
        pantalla.blit(pantallaNiveles1,(0,0))
    
    elif nivel >= 4 and nivel < 7:
        pantalla.blit(pantallaNiveles2,(0,0))
            
    elif nivel >= 7 and nivel < 10:
        pantalla.blit(pantallaNiveles3,(0,0))
            
    else:
        pantalla.blit(pantallaNivelFinal,(0,0))

def reproducirMusicaMenu():
    """
    Reproduce la música del menú principal.
    """
    pygame.mixer.music.load("Himno Universidad de Santiago De Chile.mp3")
    pygame.mixer.music.play(-1)
    
def guardarPuntaje(puntaje):
    """
    Funcion que guardara el puntaje en un archivo de texto
    El cual sera ordenado del mayor a menor

    Parametros:
    -----------
    puntaje (Int) = Puntaje del jugador hasta el momento
    """
    try:
        with open("puntajes.txt", "r") as archivo:
            puntajes=archivo.readlines()
    except FileNotFoundError:
        puntajes=[]

    puntajes=[int (p.strip()) for p in puntajes]
    puntajes.append(puntaje)

    puntajes=sorted(puntajes,reverse=True)[:5]

    with open("puntajes.txt", "w") as archivo:
        for p in puntajes:
            archivo.write(f"{p}\n")

def mostrarTopPuntajes(pantalla):
    """
    Funcion que mostrara el Top de mejores puntajes hasta el momento:
    
    Parametros:
    -------------
    pantalla = Variable que representa la pantalla del videojuego
    """
    
    try:
        with open("puntajes.txt", "r") as archivo:
            puntajes=archivo.readlines()
    except FileNotFoundError:
        puntajes=[]

    puntajes=[int(p.strip()) for p in puntajes]

    pantalla.fill((0,0,0))  
    fuente=pygame.font.Font("Silkscreen-Bold.ttf",48)
    mensaje=fuente.render("Top de Puntajes",True,(255,255,255))
    pantalla.blit(mensaje,(150,50))

    for i, puntaje in enumerate(puntajes):
        texto=fuente.render(f"{i + 1}. {puntaje}",True,(255,255,255))
        pantalla.blit(texto,(300, 150 + i*50))

    pygame.display.update()
    pygame.time.delay(5000)  


## Ejecucion

In [6]:
#Se hace la configuración inicial.
clock=pygame.time.Clock()
nave=Jugador()
meteoritos=pygame.sprite.Group()
puntaje=0
ultimoTiempo=time.time()
jefe=JefeFinal()

#Se crean las variables booleanas para utilizar durante la ejecución.
ejecucion=True
ingresarAjustes=False
inicioJuego=False
escucha=True
musica=True
opcionesDerrota=True

#Se crean las variables para el manejo de los niveles del juego.
nivelActual=3
enemigosDerribados=0
nivelCambiado=False
tiempoEsperaNivel=2000
ultimoTiempoNivel=time.time()

#Se crean variables para controlar la velocidad entre cada disparo.
intervaloDisparo=0.05
ultimoDisparo=time.time()

#Se hace un ciclo para ejecutar el juego.
while ejecucion:
    pantalla.fill((0,0,0))
    for opcionesJuego in pygame.event.get():
        if opcionesJuego.type == pygame.QUIT: 
            ejecucion=False

        if opcionesJuego.type == pygame.MOUSEBUTTONDOWN:
            if ajustes.collidepoint(opcionesJuego.pos): 
                ingresarAjustes = True

            if inicio.collidepoint(opcionesJuego.pos) and ingresarAjustes == False: 
                inicioJuego=True
                if musica == True:
                    reproducirMusica(nivelActual)
                else:
                    pygame.mixer.music.stop()

        if opcionesJuego.type == pygame.KEYDOWN:
            if opcionesJuego.key == pygame.K_SPACE: 
                if time.time()-ultimoDisparo >= intervaloDisparo:
                    nave.disparar()
                    
                    if escucha == True: 
                        sonidoDisparo.play(loop =0)
                    else:
                        sonidoDisparo.stop()
   
                    ultimoDisparo=time.time()
                        
    #Se crea un condicional con el apartado de opciones de la pantalla de juego en los niveles.
    if inicioJuego:
        cambiarPantallaNiveles(nivelActual) 
        teclas=pygame.key.get_pressed() 
        nave.mover(teclas) 
        nave.aparicion(pantalla) 
        nave.mostrarBarraVida(pantalla)  
        nave.mostrarBarraEscudo(pantalla)

        meteoritos,crearMeteoritos,contadorDeMeteoritos,nivelActual,nivelCambiado,jefeActivo,mejoraVidaActivo,mejoraEscudoActivo,mejoraDañoActivo = enemigosPorNivel(nivelActual,
                                                    meteoritos,crearMeteoritos,tiempoDeCreacion,cantidadDeMeteoritos,contadorDeMeteoritos,nave,nivelCambiado,
                                                    jefe,jefeActivo,mejoraVidaActivo,mejoraEscudoActivo,mejoraDañoActivo
                                                    )
        # Se solicita las funciones de cada instancia dependiendo de su clase
        for enemigo in meteoritos:
            if isinstance(enemigo,Enemigos):
                enemigo.caer()  
            elif isinstance(enemigo,JefeFinal):
                
                puntaje=nave.actualizarJefe(puntaje,enemigo,jefeActivo)
                enemigo.mover()  
                enemigo.disparar()  
                enemigo.actualizarProyectiles(pantalla)  
            elif isinstance(enemigo,Mejora):
                enemigo.caer()
                nave.actualizarMejora(enemigo)
        
            enemigo.mostrar(pantalla)  

        #Se configura la puntuación.
        puntaje=nave.actualizar(meteoritos,puntaje)
            
        if time.time()-ultimoTiempo >= 1: 
            puntaje+=1
            ultimoTiempo=time.time() 

        #Se crea un condicional para realizar cambios por cada nivel superado.
        if nivelCambiado == True:
            #Se muestra una pantalla que indica el fin de un nivel y da paso al siguiente.
            mostrarMensaje(pantalla,f"Nivel {nivelActual-1} superado",(255,255,255),400,300,font_size=48)
            pygame.display.update()
            pygame.time.delay(tiempoEsperaNivel)

            #Cambiar música al nuevo nivel.
            if musica == True:
                reproducirMusica(nivelActual)
            nivelCambiado=False

        mostrarPuntaje(pantalla,puntaje) 
        pygame.display.update() 
        
        if nave.vida == 0:
            nave.velocidad=5  
            intervaloDisparo=0.4 

        # Se muestra una pantalla al ganar al juego    
        if jefeActivo and jefe.disparosRecibidos >= jefe.disparosNecesarios:
            juegoTerminado=True
            while juegoTerminado:
                if not pygame.get_init():
                    break  
        
                mostrarMensaje(pantalla,f"¡Superaste el juego!",(255,255,255),400,300,font_size=50)
                pygame.display.update()
                pygame.time.delay(2000)
        
                guardarPuntaje(puntaje)

                mostrarTopPuntajes(pantalla)
        
                opcionesFinal=True
                while opcionesFinal:
                    if not pygame.get_init():
                        break 
        
                    mostrarMensaje(pantalla,"Presiona R para Reiniciar o Q para Salir",(255,255,255),400,300,font_size=20)
                    pygame.display.update()
        
                    for evento in pygame.event.get():
                        if evento.type == pygame.QUIT:
                            pygame.quit()
                            exit()
        
                        if evento.type == pygame.KEYDOWN:
                            if evento.key == pygame.K_q:
                                nave=Jugador()
                                meteoritos.empty()
                                puntaje=0
                                inicioJuego=False  
                                nivelActual=1
                                mejoraVidaActivo=False
                                mejoraEscudoActivo=False
                                mejoraDañoActivo=False
                                jefeActivo=False
                                juegoTerminado=False
                                pygame.mixer.music.stop()
                                if musica:
                                    reproducirMusicaMenu()
                                opcionesFinal=False
                                
                            elif evento.key == pygame.K_r:
                                nave=Jugador()
                                meteoritos.empty()
                                puntaje=0
                                inicioJuego=True
                                jefeActivo=False
                                nivelActual=1
                                mejoraVidaActivo=False
                                mejoraEscudoActivo=False
                                mejoraDañoActivo=False
                                juegoTerminado=False
                                pygame.mixer.music.stop()
                                if musica:
                                    reproducirMusica(nivelActual)
                                opcionesFinal=False

        # Si el jugador pierde, se muestra una pantalla de game over                       
        if nave.escudo <= 0:
            reiniciarNiveles,salirNiveles=pantallaPerdida()
            pygame.mixer.music.stop()
            pygame.display.update()

            opcionesDerrota=True
            while opcionesDerrota:
                for evento in pygame.event.get():
                    if evento.type == pygame.MOUSEBUTTONDOWN:
                        if salirNiveles.collidepoint(evento.pos):
                            inicioJuego=False
                            ingresarAjustes=False
                            puntaje=0
                            nave=Jugador()
                            jefe=JefeFinal()
                            jefeActivo=False
                            mejoraVidaActivo=False
                            mejoraEscudoActivo=False
                            mejoraDañoActivo=False
                            intervaloDisparo=0.05
                            meteoritos.empty()
                            
                            pygame.mixer.music.stop()
                            if musica:
                                reproducirMusicaMenu()
                            opcionesDerrota=False
                            
                        elif reiniciarNiveles.collidepoint(evento.pos):
                            nave=Jugador()
                            jefe=JefeFinal()
                            intervaloDisparo=0.05
                            meteoritos.empty()
                            jefeActivo=False
                            mejoraVidaActivo=False
                            mejoraEscudoActivo=False
                            mejoraDañoActivo=False
                            puntaje=0
                            inicioJuego=True
                            nivelActual=1
                            
                            pygame.mixer.music.stop()
                            if musica:
                                reproducirMusica(nivelActual)
                            opcionesDerrota=False

    # Se crea un condicional con el apartado de opciones de la pantalla de ajustes del juego.
    if ingresarAjustes:
        pantallaAjustes()
        salirAjustes,onMusica,offMusica,onSonido,offSonido=pantallaAjustes()
        pygame.display.update()  

        for opcionesAjustes in pygame.event.get():
            if opcionesAjustes.type == pygame.MOUSEBUTTONDOWN:
                if offMusica.collidepoint(opcionesAjustes.pos):
                    pygame.mixer.music.stop()
                    musica=False 

                if onMusica.collidepoint(opcionesAjustes.pos):
                    if pygame.mixer.music.get_busy() == False:
                        pygame.mixer.music.play(-1)
                        musica=True

                if offSonido.collidepoint(opcionesAjustes.pos):
                    escucha=False 
                    sonidoJefeActivo=False

                if onSonido.collidepoint(opcionesAjustes.pos):
                    escucha=True 

            if opcionesAjustes.type == pygame.QUIT:
                ejecucion=False

            if opcionesAjustes.type == pygame.MOUSEBUTTONDOWN:
                if salirAjustes.collidepoint(opcionesAjustes.pos):
                    ingresarAjustes=False
                    
    #Se crea un condicional para volver a la pantalla de inicio.
    if (ingresarAjustes == False) and (inicioJuego == False):
        pantalla.blit(pantallaDeInicio,[0,0])
        pantalla.blit(textoTitulo,textoRect1)

        pygame.draw.rect(pantalla,(160,32,240),inicio)
        pantalla.blit(textoInicio,textoRect2)

        pygame.draw.rect(pantalla,(160, 32, 240),ajustes)
        pantalla.blit(textoAjustes,textoRect3)

    pygame.display.flip() 
    clock.tick(60) 

pygame.quit()